In [6]:
import pandas as pd

# data = {
#     '搜尋詞': ['淋浴水龍頭']*10,
#     'Rank': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
#     'tf-idf': [
#         'JAVA 雙頭淋浴水龍頭 TLS414, 1組', 
#         'MIRINAE 壁掛式淋浴水龍頭 MF-102, 1組', 
#         'American Standard 美標 淋浴水龍頭 FB2689, 1個', 
#         'DAELIM BATH 洗臉台淋浴水龍頭 DL-B1615, 1組', 
#         'HANSSEM 淋浴方形水龍頭 HI-9030', 
#         '高級水龍頭 黑色, 1個', 
#         '高級水龍頭 藍色, 1個', 
#         'YJJ 水龍頭過濾器3.0型水槽旋轉水龍頭龍頭套裝, 1組', 
#         '水龍頭全浴美容洗髮水台水龍頭TPF-0105, 1套', 
#         '我們的水早晨壁掛式單桿淋浴浴缸水龍頭, 1個'],
#     'ner_relevancy_1': [0] * 10,
#     'semantic_model': [
#         '淋浴水龍頭, 1入',
#         '淋浴水龍頭組 黑色, 1個', 
#         '趨勢淋浴豪華浴室水龍頭鍍鉻光澤 RA-600, 1個', 
#         '藍水浴室壁掛式淋浴龍頭 2-600, 1個', 
#         'Jeilbas 鍍鉻鍍金浴室淋浴龍頭套組, 1套', 
#         '水龍頭龍頭兩孔頂級水槽管道淋浴龍頭EINS-3033, 1個', 
#         '水壓上升節水淋浴噴頭銀色, 1個', 
#         '淋浴龍頭 T-02, 1個', 
#         '壁掛式浴室淋浴龍頭金屬 OV3-600, 1個', 
#         '淋浴龍頭 GK-102, 1個'],
#     'ner_relevancy_2': [0] * 10
# }

# df = pd.DataFrame(data)
# df

path = 'Mxxxxxxxx_搜尋比較.xlsx'
student_id = path.split('_')[0]

df = pd.read_excel(path)
df

,搜尋詞,Rank,tf-idf,ner_relevancy_0,semantic_model,ner_relevancy_1
0,分類垃圾桶,1,"淋浴球 M02, Type02, 5件, 1組",NaN,"分類垃圾桶, 藍色, 1個",NaN
1,分類垃圾桶,2,"Shushuroom棉麻花布SH-325, 1入, 4號",NaN,"拼色雙格分類垃圾桶 L號, 1個, 綠色",NaN
2,分類垃圾桶,3,"Hi Who Weather 系列 Snow Daily 圓形 17 領短袖 T 卹, 淺藍",NaN,"方形分類垃圾桶 3格, 混色, 1個",NaN
3,分類垃圾桶,4,"VENTION 威迅 海水銀4K HDMI 2.0電纜AV電纜E.3M（AABII）, 1個...",NaN,"拼色雙格分類垃圾桶 S號, 綠色, 1個",NaN
4,分類垃圾桶,5,"COBI 小型陸軍汽車座 2372, 混色",NaN,"拼色雙格分類垃圾桶 S號, 藍色, 1個",NaN
...,...,...,...,...,...,...
2495,急救箱,6,"Yako Soft Yuto 100g x 6p + 抹刀 3 件套, 混色, 1組",NaN,"北歐設計整潔家用急救箱粉色M, 1個",NaN
2496,急救箱,7,"造型車貼組 25*20cm, 白色的, 1個",NaN,"便攜藥箱急救箱收納急救箱透明+紅色, 2個",NaN
2497,急救箱,8,"House Of Seoha 檸檬精油, 10ml, 5個",NaN,"安全急救箱粉色, 5個",NaN
2498,急救箱,9,"DESIGN SYNDROME 鬍鬚花環, 鬍鬚（黑色）",NaN,"家庭急救箱, 1個",NaN


In [7]:
from src import inference_api
import torch, src.config as config

# all_attribute = ['品牌', '名稱', '產品', '產品序號', '顏色', '材質', '對象與族群', '適用物體、事件與場所', 
#                      '特殊主題', '形狀', '圖案', '尺寸', '重量', '容量', '包裝組合', '功能與規格']

# set device
config.string_device = 'cuda' if torch.cuda.is_available() else 'cpu'
config.device = torch.device(config.string_device)

# load model
model, tokenizer = inference_api.load_model("clw8998/Product-Name-NER-model", device=config.device)

# Relevancy Calculation Function: Handles the comparison and calculation of relevancy scores
def ner_relevancy(df, index, all_results, check_att, margin):
    query_tags_dict = all_results[df.loc[index, '搜尋詞'].lower()]
    tfidf_tags_dict = all_results[df.loc[index, 'tf-idf'].lower()]
    semantic_tags_dict = all_results[df.loc[index, 'semantic_model'].lower()]

    query_tags_pool = set(ent[0] for ents in query_tags_dict.values() for ent in ents if ent)
    tfidf_tags_pool = set(ent[0] for ents in tfidf_tags_dict.values() for ent in ents if ent)
    semantic_tags_pool = set(ent[0] for ents in semantic_tags_dict.values() for ent in ents if ent)

    # 計算 query tags 與 tf-idf 的 tags 相關性
    if len(query_tags_pool & tfidf_tags_pool) >= len(check_att) * margin:
        df.loc[index, 'ner_relevancy_0'] = 2
    elif len(query_tags_pool & tfidf_tags_pool) >= len(check_att) * margin * 0.5:
        df.loc[index, 'ner_relevancy_0'] = 1
    else:
        df.loc[index, 'ner_relevancy_0'] = 0

    # 計算 query tags 與 semantic model 的 tags 相關性
    if len(query_tags_pool & semantic_tags_pool) >= len(check_att) * margin:
        df.loc[index, 'ner_relevancy_1'] = 2
    elif len(query_tags_pool & semantic_tags_pool) >= len(check_att) * margin * 0.5:
        df.loc[index, 'ner_relevancy_1'] = 1
    else:
        df.loc[index, 'ner_relevancy_1'] = 0


    return df

In [8]:
check_att = ['品牌', '產品', '顏色', '適用物體、事件與場所', '功能與規格']
all_results = inference_api.get_ner_tags(
        model, 
        tokenizer, 
        list(set(df['搜尋詞'].tolist() + df['tf-idf'].tolist() + df['semantic_model'].tolist())), 
        check_att)

100%|██████████| 697/697 [00:19<00:00, 36.59it/s]


In [9]:
for index, row in df.iterrows():
    df = ner_relevancy(df, index, all_results, check_att, 0.3)
df

,搜尋詞,Rank,tf-idf,ner_relevancy_0,semantic_model,ner_relevancy_1
0,分類垃圾桶,1,"淋浴球 M02, Type02, 5件, 1組",0.0,"分類垃圾桶, 藍色, 1個",2.0
1,分類垃圾桶,2,"Shushuroom棉麻花布SH-325, 1入, 4號",0.0,"拼色雙格分類垃圾桶 L號, 1個, 綠色",2.0
2,分類垃圾桶,3,"Hi Who Weather 系列 Snow Daily 圓形 17 領短袖 T 卹, 淺藍",0.0,"方形分類垃圾桶 3格, 混色, 1個",2.0
3,分類垃圾桶,4,"VENTION 威迅 海水銀4K HDMI 2.0電纜AV電纜E.3M（AABII）, 1個...",0.0,"拼色雙格分類垃圾桶 S號, 綠色, 1個",2.0
4,分類垃圾桶,5,"COBI 小型陸軍汽車座 2372, 混色",0.0,"拼色雙格分類垃圾桶 S號, 藍色, 1個",2.0
...,...,...,...,...,...,...
2495,急救箱,6,"Yako Soft Yuto 100g x 6p + 抹刀 3 件套, 混色, 1組",0.0,"北歐設計整潔家用急救箱粉色M, 1個",1.0
2496,急救箱,7,"造型車貼組 25*20cm, 白色的, 1個",0.0,"便攜藥箱急救箱收納急救箱透明+紅色, 2個",1.0
2497,急救箱,8,"House Of Seoha 檸檬精油, 10ml, 5個",0.0,"安全急救箱粉色, 5個",1.0
2498,急救箱,9,"DESIGN SYNDROME 鬍鬚花環, 鬍鬚（黑色）",0.0,"家庭急救箱, 1個",0.0


In [10]:
df.to_csv(f'{student_id}-NER-relevancy.csv', index=False, encoding='utf-8-sig')